In [1]:
import matplotlib.pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
from sklearn import datasets, svm
from sklearn.model_selection import train_test_split
import predict as ka
import sys
import logistic_regression
from logistic_regression import gradient_descent

/home/daniel/.local/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Data Handling

In [2]:
df = pd.read_csv("SPECTF.csv")
def accuracy_score(Y_true, Y_predict):
    error=0
    for i in range(len(Y_predict)):
        if Y_true[i] != Y_predict[i]:
            error = error + 1
    error= float(error)/len(Y_predict)
    return 1-error

In [3]:
formula = 'string0 ~ string1 + string2 + string3 + string4 + string5 + string6 + string7 + string8 + string9 + string10 + string11 + string12 + string13 + string14 + string15 + string16 + string17 + string18 + string19 + string20 + string21 + string22 + string23 + string24 + string25 + string26 + string27 + string28 + string29 + string30 + string31 + string32 + string33 + string34 + string35 + string36 + string37 + string38 + string39 + string40 + string41 + string42 + string43 + string44'
results = {}

### Logistic Regression

In [4]:
print("\tAccuracy")
print("Lasagne"+"\t\t\t"+"Me")

for i in range(5):
    class NullWriter(object):
        def write(self, arg):
            pass
    
    oldstdout = sys.stdout
    sys.stdout = NullWriter()
    
    train, test = train_test_split(df, test_size = 0.2)
    y,x = dmatrices(formula, data=train, return_type='dataframe')
    model = sm.Logit(y,x)
    res = model.fit()
    results['Logit'] = [res, formula]

    compared_resuts = ka.predict(test, results, 'Logit')
    compared_resuts = Series(compared_resuts)
    for k in range(compared_resuts.size):
        if compared_resuts[k]>0:
            compared_resuts[k]=1
        else:
            compared_resuts[k]=0
    compared_resuts=np.array(compared_resuts)
    true=np.array(test.iloc[:,1])

    log_acc=0
    for j in range(compared_resuts.size):
        if compared_resuts[j]==true[j]:
            log_acc=log_acc+1
    
    sys.stdout = oldstdout
    
    yt, xt = dmatrices(formula, data=test, return_type='dataframe')
    yt = np.asarray(yt).ravel()
    
    x_feat=np.column_stack((np.ones(np.array(x)[:,1:].shape[0]),np.array(x)[:,1:]))
    x_feat_test=np.column_stack((np.ones(np.array(xt)[:,1:].shape[0]),np.array(xt)[:,1:]))
    
    y = np.asarray(y).ravel()
    beta=gradient_descent(x_feat,y)
    
    y_pred=x_feat_test.dot(beta)
    for n in range(y_pred.size):
        if y_pred[n]>0:
            y_pred[n]=1
        else:
            y_pred[n]=0
    
    print(str(float(log_acc)/compared_resuts.size)+"\t\t"+str(accuracy_score(yt,y_pred)))

	Accuracy
Lasagne			Me
0.87037037037		0.759259259259
0.62962962963		0.814814814815
0.796296296296		0.759259259259
0.814814814815		0.722222222222
0.851851851852		0.759259259259


### Random Forest

In [5]:
import sklearn.ensemble as ske
import random_forest
from random_forest import RandomForest



print("\tAccuracy")
print("Lasagne"+"\t\t"+"Me")
for i in range(5):
    train, test = train_test_split(df, test_size = 0.2)
    y, x = dmatrices(formula, data=train, return_type='dataframe')
    y = np.asarray(y).ravel()
    
    results_rf = ske.RandomForestClassifier(n_estimators=100).fit(x, y)
    resilts_rf_my=RandomForest(10,80)
    resilts_rf_my.fit(np.array(x))


    yt, xt = dmatrices(formula, data=test, return_type='dataframe')
    yt = np.asarray(yt).ravel()

    predict_my=resilts_rf_my.predict(np.array(xt))

    score = results_rf.score(xt, yt)
    print "{}\t{}".format(score,accuracy_score(yt,[row[0] for row in predict_my]))

	Accuracy
Lasagne		Me
0.796296296296	0.759259259259
0.814814814815	0.833333333333
0.814814814815	0.833333333333
0.851851851852	0.777777777778
0.87037037037	0.796296296296
